### Step 1: Prepare documents

In [16]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)
        

In [17]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [18]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

### Step 2: Create Embeddings using Pretrained Models

In [19]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')

In [12]:
len(model.encode('This is a simple sentence'))

768

In [20]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [21]:
from tqdm.autonotebook import tqdm

operations = []

for doc in tqdm(documents):
    doc['text_vector'] = list(model.encode(doc['text']))
    operations.append(doc)

  0%|          | 0/948 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
operations[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.03570346,
  -0.06891429,
  -0.044483423,
  0.0062501533,
  -0.042247094,
  -0.0064539956,
  0.027548144,
  -0.018610273,
  -0.052037504,
  -0.0024047394,
  0.0574545,
  -0.0155926,
  0.015488839,
  0.0987016,
  0.015649557,
  -0.061262548,
  0.04107641,
  0.021717312,
  -0.09465402,
  0.0032483656,
  0.0044765803,
  -0.0076589384,
  -0.05192878,
  -0.007335758

### Step 3: Setup ElasticSearch connection

In [ ]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [ ]:
es_client.info()

ObjectApiResponse({'name': 'e694d3f0e280', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'BvdT1zIlTKyWHgBw6tIInA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Step 4: Create Mappings and Index

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [24]:

index_name = "course-questions"

# es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

#### Step 5: Add documents into index

In [25]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

#### Step 6: Create end user query

In [41]:
search_term = "windows or mac?"

vector_search_term = model.encode(search_term)

In [27]:
query = {
    "field" : "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates":10000
}

In [28]:
res = es_client.search(index=index_name,knn=query,source=["text", "section", "question", "course"])
res['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'ZfAQopABdjXu8CpkMsDT',
  '_score': 0.7147919,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'VvAQopABdjXu8CpkMcAD',
  '_score': 0.5985866,
  '_source': {'question': 'Environment - Should I use my local machine, GCP, or GitHub Codespaces for my environment?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'You can set it up on your laptop or PC if you prefer to work locally from your laptop or PC.\nYou might face some challenges, especially for Windows users. If you face cnd2\nIf you prefer to work on the local machine, you may start with the week 1 Introduction to Docker and follow through.\nHow

#### Step 7: Perform Keyword search with Semantic Search (Hybrid/Advanced Search)

In [42]:
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [43]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"course": "data-engineering-zoomcamp"},
    },  
    knn=knn_query,
    size=5
)

In [ ]:
# response = es_client.search(
#     index = index_name,
#     query = {
#         "bool" : {
#             "must" : {
#                 "multi_match":
#                     {
#                         "query" : "windows or python",
#                         "fields": ["text", "question", "course", "title"],
#                         "type" : "best_fields"
                        
#                     }
#             },
#             "filter" : {
#                 "term" : {
#                     "course": "data-engineering-zoomcamp"
#                 }
#             }
#         }
#     }
# )

In [45]:
response["hits"]["hits"]


[{'_index': 'course-questions',
  '_id': 'ZfAQopABdjXu8CpkMsDT',
  '_score': 0.7187058,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965461671352386,
    -0.000626126304268837,
    -0.01662949100136757,
    0.05285150930285454,
    0.05476527288556099,
    -0.03133990615606308,
    0.029942581430077553,
    -0.04808562621474266,
    0.04467551037669182,
    0.005839474033564329,
    0.016233040019869804,
    0.012001154012978077,
    -0.031222281977534294,
    0.016600528731942177,
    -0.04886901378631592,
    -0.06496307998895645,
    0.046434223651885986,
    -0.009297756478190422,
    -0.0642528235912323,
    -0.01373267825692892,
    -0.015976183116436005,
    0.008629541844129562,
    -0.02447899058461